In [3]:
# @title Authentication

%pip install --upgrade --quiet google-cloud-aiplatform

#  For only colab to authenticate to get an access to the GCP.
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 56.0 MB/s eta 0:00:00


In [4]:
# @title Agent initialization.

from IPython.display import display, Markdown

import vertexai
import vertexai.generative_models as generative_models
from vertexai.preview.generative_models import grounding

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    Part,
    Tool
)

MODEL_NAME="gemini-1.5-flash"
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"

vertexai.init(project=PROJECT_ID, location=REGION)
model = GenerativeModel(MODEL_NAME)

In [7]:
# @title Helper function
from vertexai.generative_models import ChatSession, GenerationConfig

def interactive_chat(chat: ChatSession, prompt: str, search:bool) -> str:

    tool = None
    if search:
      tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())

    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 1,
        "top_p": 0.95,
        "top_k": 40
    }

    tools = [tool] if search else None

    safety_settings = {
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    }

    responses = chat.send_message(
        prompt,
        generation_config = generation_config,
        safety_settings=safety_settings,
        tools = tools,
        stream=False
    )
    return responses.text

#------------------------------------------------------------------------------------

def get_chat_history(chat):

  history = " ".join([content.text for content in chat.history])
  return history

#------------------------------------------------------------------------------------
def query_classification(chat_history, query):

  prompt = f"""당신은 이전 대화 내용을  참조하여 <입력질문>을 분류해주는 AI Assistant 입니다.
  1. 질문 분류를 기준은 아래 <분류기준>을 참고하여 아래 내용을 포함된 정보로 분류해주세요.
     - 분류이유
     - 분류결과
     - Endpoint

  2. 질문내의 키워드를 추출해서 정리해주세요.

  <대화내용>{chat_history}</대화내용>
  <입력질문>{query}</입력질문>

  <질문분류기준> [분류기준:endpoint]
  1. 제품 추천 : http://sec_vd.com/제품검색
  2. 대리점 추천 : http://sec_vd.com/대리점추천
  3. 번역테스트 : http://sec_vd.com/번역테스트
  4. 대리점 수리접수 : http://sec_vd.com/대리점수리접수
  5. 기타 : http://sec_vd.com/기타

  """

  response_schema = {
      "type": "OBJECT",
      "properties": {
        "reason": {"type": "STRING"},
        "classification": {
            "type": "OBJECT",
            "properties": {
              "class": {"type": "STRING"},
              "endpoint": {"type": "STRING"},
            },
        }
      }
  }

  generation_config = GenerationConfig(
        temperature=0,
        top_p=1.0,
        top_k=10,
        candidate_count=1,
        max_output_tokens=8192,
        response_mime_type="application/json",
        response_schema=response_schema

    )

  responses = model.generate_content(
      [prompt],
      generation_config=generation_config,
      stream=False,
  )

  return responses.text

#------------------------------------------------------------------------------------

def get_prompts(chat_history, query, location):

  prompt_1 =f"""
          당신은 삼성 제품 추천을 위한 AI 어시스턴트 입니다. 답변은 아래 내용을 따라서 답해주세요.

          1. 삼성 제품 추천에 대해서 제품명 2~3개와 이유를 2줄 이내로 먼저 답변 해주세요.
          2. 추천을 하는 지역위치는 최신 대화의 지역을 우선으로 추천해주세요.
          3. 질문 또는 대화내용에서 특정 위치를 알려주지 않으면 기본값으로 {location} 지역을 기반으로 답해주세요.
          4. 사전 훈련된 지식에서 질문하는 지역에서 자주 언급되는 제품을 이용해서 추천해주시고, 이유를 알려주세요.
          5. 자세한 답변을 위해서 추가적으로 사용자의 관심사항(색상,크기,기능등을) 요청해주세요.
          6. 전체적인 답변은 간략하게 최대 10줄을 넘지 않게 해주세요.

          <질문> : {query}

  """

  prompt_2 = f"""당신은 삼성 가전 제품 사용자를 위해 번역해주는 AI 어시스턴트 입니다. 아래 내용에 맞게 번역해주세요.

          1. 번역 대상 언어 선택은 가장 최근 대화를 참고해서 선택하고, 없다면 번역대상 언어 지정을 사용자에게 요청해주세요.
          2. 번역을 해야할 부분은 요청하는 문맥에 맞게 선택해서 번역해주세요.
          3. 특별하게 번역해야 할 부분을 제시하지 않으면 전체 문장을 번역해주세요.
          4. 답변은 최대한 간략하게 번역된 부분만 표현하고 설명하지 말아주세요.

          <번역을 위한 질문> : {query}

  """

  prompt_3 = f"""당신은 삼성 제품 사용자를 위해 삼성 대리점 위치 추천해주는 AI Assistant입니다.

          1. 질문에 맞는 삼성제품을 구매할 수 있는 대리점 명을 3~4개 정도 추천해주세요.
          2. 해당 대리점의 주소를 알려주고 왜 추천하는지 알려주세요.
          3. 전체적인 답변은 최대 10줄을 넘지 않게 해주세요.

          <질문> : {query}

  """

  prompt_4 = f"""당신은 삼성제품 사용자를 위해 대리점 수리 접수를 도와주는 AI Assistant입니다.

          1. 수리접수를 위한 수리점 위치는 최신 대화의 지역을 우선으로 구글에서 검색해서 추천해주세요.
          2. 질문 또는 대화내용에서 특정 위치를 알려주지 않으면 기본값으로 {location} 지역을 기반으로 답해주세요.
          3. 수리점의 정확한 주소와 전화번호를 같이 보여주세요.
          4. 만일 정확한 지역정보 또는 수리점 위치를 모를 때는 해당 위치정보를 요청해주세요.
          5. 전체적인 답변은 간략하게 최대 10줄을 넘지 않게 해주세요.

          <질문> : {query}

  """

  prompt_5 = f"""당신은 삼성제품 사용자의 질문에 답변을 해주는 AI Assistant입니다.
          사용자의 다양한 질문에 대해서 검색 결과를 활용해서 답변 해주세요.
          가급적 답변은 10줄 이내로 짧고 간략하게 답해주세요.
          <질문> : {query}

  """

  classified = query_classification(chat_history, query)

  task = eval(classified)['classification']['class']

  if task == '제품 추천':
    return prompt_1, True
  elif task == '번역테스트':
    return prompt_2, False
  elif task == '대리점 추천':
    return prompt_3, True
  elif task == '대리점 수리접수':
    return prompt_4, True
  elif task == '기타':
    return prompt_5, True
  else:
    return prompt_5, True


In [8]:
# @title AI Agent

chat = model.start_chat()

location = "경기 과천"
search = True

while True:
  query = input('사용자: ')

  if query == '종료': break

  chat_history = get_chat_history(chat)

  prompt, search = get_prompts(chat_history, query, location)

  response = interactive_chat(chat, prompt, search)
  display(Markdown(f"AI Agent : {response}"))
  print(f"------------------------------------ ")

사용자: 제품 좋은거 추천해주세요.


AI Agent : 과천 지역에서 자주 언급되는 삼성 제품으로는 비스포크 냉장고와 갤럭시 탭 S9가 있습니다. 비스포크 냉장고는 다양한 색상과 디자인으로 주방 인테리어에 잘 어울리고, 갤럭시 탭 S9는 뛰어난 성능과 휴대성으로 업무와 엔터테인먼트에 모두 유용하게 사용할 수 있습니다.

비스포크 냉장고는 어떤 종류의 냉장고를 원하시나요? 4도어, 1도어, 혹은 다른 종류의 냉장고를 선호하시나요? 혹시 원하는 색상이나 크기가 있으신가요?

갤럭시 탭 S9는 어떤 용도로 사용하실 계획이신가요? 업무용, 학업용, 혹은 엔터테인먼트용으로 사용하시나요? 혹시 원하는 저장 용량이나 색상이 있으신가요? 


------------------------------------ 
사용자: 어디서 사면 되나요 ?


AI Agent : 과천 지역에서 삼성 제품을 구매할 수 있는 대리점으로는 KT 과천 스마트타워, SK 과천 청사점, 그리고 삼성 디지털프라자 과천점을 추천합니다.

KT 과천 스마트타워는 과천시 중앙동에 위치하고 있으며, KT에서 직접 운영하는 대리점으로 다양한 삼성 제품을 구매할 수 있습니다. SK 과천 청사점은 과천시 별양동에 위치하고 있으며, 고객 만족도가 높은 매장으로 알려져 있습니다. 삼성 디지털프라자 과천점은 과천시 별양동에 위치하고 있으며, 삼성 제품을 전문적으로 판매하는 대리점입니다. 

어떤 제품을 구매하시려는지 알려주시면 더 자세한 정보를 드릴 수 있습니다. 


------------------------------------ 
사용자: 별양동에 있는 지점이 뭐라고요 ?


...

------------------------------------ 
사용자: 수원지역도 좀 알려줄래 ?


AI Agent : 수원 지역에서 삼성 제품을 구매할 수 있는 대리점으로는 삼성 디지털프라자 수원 권선점, 삼성 디지털프라자 수원 영통점, 그리고 삼성스토어 북수원을 추천합니다.

삼성 디지털프라자 수원 권선점은 수원시 권선구 권선동에 위치하고 있으며, 다양한 삼성 제품을 한곳에서 만나볼 수 있는 대형 매장입니다. 삼성 디지털프라자 수원 영통점은 수원시 영통구 영통동에 위치하고 있으며, 갤럭시 제품을 전문적으로 판매하는 매장입니다. 삼성스토어 북수원은 수원시 장안구 영화동에 위치하고 있으며, 삼성 제품을 전문적으로 판매하는 대리점입니다. 

어떤 제품을 구매하시려는지 알려주시면 더 자세한 정보를 드릴 수 있습니다. 


------------------------------------ 
사용자: 아니 서비스 센터.


AI Agent : 수원 지역의 삼성 서비스센터는 다음과 같습니다.

* 삼성전자서비스 북수원센터
    * 주소: 경기 수원시 장안구 영화동 104-28번지 삼성스토어 북수원 2층
    * 전화번호: 1588-3366

* 삼성전자서비스 호매실휴대폰센터
    * 주소: 경기 수원시 권선구 구운동 282-1 ( 2층 ) 
    * 전화번호: 1588-3366

제품 수리접수를 위해서는 해당 서비스센터에 전화하여 예약 후 방문하시면 됩니다. 


------------------------------------ 
사용자: 종료
